In [ ]:
# 필요한 패키지 import
import pandas as pd
import numpy as np

# 랜덤시드 고정
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [ ]:
# 데이터 불러오기
train = pd.read_csv('../input/2022-ml-finalexam-p3/train.csv')
test = pd.read_csv('../input/2022-ml-finalexam-p3/test.csv')

In [ ]:
# 각각 메뉴 모두 합쳐 리스트에 담기
breakfast_train = [x for x in train['조식메뉴']]
lunch_train = [x for x in train['중식메뉴']]
dinner_train = [x for x in train['석식메뉴']]

breakfast_test = [x for x in test['조식메뉴']]
lunch_test = [x for x in test['중식메뉴']]
dinner_test = [x for x in test['석식메뉴']]

In [ ]:
# 필요 없는 column drop 및 라벨 분리
x = train.drop(['Unnamed: 0', '일자', '조식메뉴', '중식메뉴', '석식메뉴', '중식계'], axis=1)
y = train['중식계']
test = test.drop(['Unnamed: 0', '일자', '조식메뉴', '중식메뉴', '석식메뉴'], axis=1)

In [ ]:
# CountVectorizer 사용하여 1d 벡터화
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
vec.fit(breakfast_train + breakfast_test)
breakfast_train_feat = pd.DataFrame(vec.transform(breakfast_train).toarray().flatten())
breakfast_test_feat = pd.DataFrame(vec.transform(breakfast_test).toarray().flatten())

vec.fit(lunch_train + lunch_test)
lunch_train_feat = pd.DataFrame(vec.transform(lunch_train).toarray().flatten())
lunch_test_feat = pd.DataFrame(vec.transform(lunch_test).toarray().flatten())

vec.fit(dinner_train + dinner_test)
dinner_train_feat = pd.DataFrame(vec.transform(dinner_train).toarray().flatten())
dinner_test_feat = pd.DataFrame(vec.transform(dinner_test).toarray().flatten())

In [ ]:
# 새로운 column에 대입
x['조식'] = breakfast_train_feat
x['중식'] = lunch_train_feat
x['석식'] = dinner_train_feat

test['조식'] = breakfast_test_feat
test['중식'] = lunch_test_feat
test['석식'] = dinner_test_feat

In [ ]:
# 문자열 데이터 데이터 실수화
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
def label_encode(train, test, columns):
    for col in columns:
        labels = pd.concat([train[col], test[col]])
        le.fit(labels)
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])
        
label_encode(x, test, ['요일'])

In [ ]:
# 모델 학습 및 결과 도출
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=1)
model.fit(x, y)
res = model.predict(test)
submit = pd.read_csv('../input/2022-ml-finalexam-p3/sample_submit.csv')
submit['중식계'] = res
submit.to_csv('submit2.csv', index=False)